In [1]:
import glob
import bedwriter
import subprocess

import numpy as np
import pandas as pd

In [2]:
tsv_filename = "/home/resecmo/vigg/release_Zanthar/release_dump/TF/AGO2_HUMAN.tsv"
ago2 = pd.read_csv(tsv_filename, sep='\t', usecols=[0, 1, 14, 15])
ago2.head(5)

,#chr,pos,fdrp_bh_ref,fdrp_bh_alt
0,chr1,2406687,0.696739,0.579110
1,chr1,2506456,0.617020,0.652714
2,chr1,2506868,0.719304,0.627044
3,chr1,2506954,0.528243,0.693341
4,chr1,2507389,0.088303,0.989680


In [9]:
def copy_significant_snvs(src_files, dst_dir, p_thr=0.05, keep_counts=None):
    if keep_counts != [] and keep_counts is not None:
        raise ValueError("keep_counts should be empty list or None")
    print(f"Copying SNVs from {'/'.join(src_files[0].split('/')[:-1])} to {dst_dir}, p_value threshold = {p_thr}")
    for i, tsv_filename in enumerate(src_files):
        if i % 50 == 0:        
            print(f"Doing {i}/{len(src_files)}: {tsv_filename.split('/')[-1]}")
        
        gene_name = tsv_filename.split('/')[-1][:-4]
        bed_filename = dst_dir + '/tfs/' + gene_name + ".bed"

        df = pd.read_csv(tsv_filename, sep='\t', usecols=[0, 1, 14, 15])

        snps = []
        for i in range(df.shape[0]):
            if ((df["fdrp_bh_ref"][i] < p_thr) or (df["fdrp_bh_alt"][i] < p_thr)):
                snps.append((df["#chr"][i], df["pos"][i]))
        tf_name = tsv_filename.split('/')[-1].split('.')[0]
        keep_counts.append((tf_name, len(snps)))
        bedwriter.write_positions_in_bed(snps, bed_filename)
        bedwriter.write_positions_in_bed(snps, dst_dir + "/all_tfs.bed", append=True, label=gene_name)
    
    #print(significant_snv_counts)

In [10]:
tsv_files = glob.glob("/home/resecmo/vigg/release_Zanthar/release_dump/TF/*")
#print(tsv_files[:3])

significant_snv_counts = []
copy_significant_snvs(tsv_files, "adastra_snps", p_thr=0.05, keep_counts=significant_snv_counts)

Copying SNVs from /home/resecmo/vigg/release_Zanthar/release_dump/TF to adastra_snps, p_value threshold = 0.05
Doing 0/1140: ZN222_HUMAN.tsv
Doing 50/1140: HNRPK_HUMAN.tsv
Doing 100/1140: MZF1_HUMAN.tsv
Doing 150/1140: ARI3A_HUMAN.tsv
Doing 200/1140: ZN674_HUMAN.tsv
Doing 250/1140: ZN697_HUMAN.tsv
Doing 300/1140: ETS2_HUMAN.tsv
Doing 350/1140: ZN423_HUMAN.tsv
Doing 400/1140: ZN623_HUMAN.tsv
Doing 450/1140: NR1H3_HUMAN.tsv
Doing 500/1140: ZN224_HUMAN.tsv
Doing 550/1140: ENL_HUMAN.tsv
Doing 600/1140: CDK12_HUMAN.tsv
Doing 650/1140: IRF5_HUMAN.tsv
Doing 700/1140: HNRPL_HUMAN.tsv
Doing 750/1140: ZN660_HUMAN.tsv
Doing 800/1140: BICRA_HUMAN.tsv
Doing 850/1140: PHX2B_HUMAN.tsv
Doing 900/1140: GATA1_HUMAN.tsv
Doing 950/1140: ZN212_HUMAN.tsv
Doing 1000/1140: ANDR_HUMAN.tsv
Doing 1050/1140: KLF9_HUMAN.tsv
Doing 1100/1140: HLF_HUMAN.tsv


In [5]:
significant_snv_counts[:10]
#tsv_files[1]

[('ZN222_HUMAN', 0),
 ('ZBT24_HUMAN', 18),
 ('CATA_HUMAN', 330),
 ('KLF5_HUMAN', 279),
 ('RBM22_HUMAN', 118),
 ('MYRF_HUMAN', 0),
 ('ZN445_HUMAN', 0),
 ('NONO_HUMAN', 17),
 ('LHX2_HUMAN', 0),
 ('HNF4G_HUMAN', 59)]

In [6]:
bed_files = glob.glob("adastra_snps/*")
bed_files[:3]

['adastra_snps/ZN443_HUMAN.bed',
 'adastra_snps/SALL3_HUMAN.bed',
 'adastra_snps/UBP7_HUMAN.bed']

In [7]:
with open('adastra_snps/ZBT24_HUMAN.bed') as f:
    print(f.readlines())

['chr1\t93909752\t93909753\n', 'chr12\t95424776\t95424777\n', 'chr17\t65057105\t65057106\n', 'chr17\t78523887\t78523888\n', 'chr2\t36352953\t36352954\n', 'chr2\t171523425\t171523426\n', 'chr4\t10171394\t10171395\n', 'chr4\t74683011\t74683012\n', 'chr4\t188505770\t188505771\n', 'chr5\t54500670\t54500671\n', 'chr5\t66494838\t66494839\n', 'chr5\t127297711\t127297712\n', 'chr5\t151924744\t151924745\n', 'chr6\t32473863\t32473864\n', 'chr6\t47474134\t47474135\n', 'chr6\t155314450\t155314451\n', 'chr8\t80107467\t80107468\n', 'chr8\t81732206\t81732207\n']


In [8]:
def find_intersections(bed_files, keep_counts=None):
    if keep_counts != [] and keep_counts is not None:
        raise ValueError("keep_counts should be empty list or None")
    if keep_counts is None:
        keep_counts = []

    cistrome_bed = "/home/resecmo/vigg/cistrome_hg38/hg38_cistrome/HNF4A_HUMAN.A.bed"

    intersection_counts = {}
    for i, bed_filename in enumerate(bed_files):
        if i % 50 == 0:
            print(f"Doing {i}/{len(bed_files)}: {bed_filename}")
        intersection_filename = "intersections/with_" + bed_filename.split('/')[-1]
        #print(intersection_filename)
        with open(intersection_filename, "bw") as output_file:
            #print(["bedtools", "intersect", "-a", cistrome_bed, "-b", bed_filename])
            intersection = subprocess.run(["bedtools", "intersect", "-a", cistrome_bed, "-b", bed_filename], capture_output=True).stdout
            output_file.write(intersection)
            #print(intersection, intersection.split(b'\n'))
            
            tf_name = bed_filename.split('/')[-1].split('.')[0]
            keep_counts.append((tf_name, len(intersection.split(b'\n')) - 1))

    print(f"{sum(count[1] for count in keep_counts)} intersections found")

In [9]:
bed_files = glob.glob("adastra_snps/*")
#print(bed_files[:3])

intersection_counts = []
find_intersections(bed_files, keep_counts=intersection_counts)

Doing 0/1140: adastra_snps/ZN443_HUMAN.bed
Doing 50/1140: adastra_snps/STA5B_HUMAN.bed
Doing 100/1140: adastra_snps/ZN776_HUMAN.bed
Doing 150/1140: adastra_snps/HXC5_HUMAN.bed
Doing 200/1140: adastra_snps/PRDM2_HUMAN.bed
Doing 250/1140: adastra_snps/MITF_HUMAN.bed
Doing 300/1140: adastra_snps/FOXO1_HUMAN.bed
Doing 350/1140: adastra_snps/PPARG_HUMAN.bed
Doing 400/1140: adastra_snps/CBP_HUMAN.bed
Doing 450/1140: adastra_snps/RING1_HUMAN.bed
Doing 500/1140: adastra_snps/CCAR2_HUMAN.bed
Doing 550/1140: adastra_snps/PMEPA_HUMAN.bed
Doing 600/1140: adastra_snps/RUNX2_HUMAN.bed
Doing 650/1140: adastra_snps/ZN619_HUMAN.bed
Doing 700/1140: adastra_snps/MYRF_HUMAN.bed
Doing 750/1140: adastra_snps/ZN629_HUMAN.bed
Doing 800/1140: adastra_snps/PBX4_HUMAN.bed
Doing 850/1140: adastra_snps/ZN513_HUMAN.bed
Doing 900/1140: adastra_snps/ZN662_HUMAN.bed
Doing 950/1140: adastra_snps/INO80_HUMAN.bed
Doing 1000/1140: adastra_snps/ZN770_HUMAN.bed
Doing 1050/1140: adastra_snps/THA_HUMAN.bed
Doing 1100/1140: ad

In [10]:
significant_snv_counts.sort()
intersection_counts.sort()
a = []
for gene_snv_count, gene_intersection_count in zip(significant_snv_counts, intersection_counts):
    gene_name = gene_snv_count[0]
    ratio = gene_intersection_count[1] / gene_snv_count[1] if gene_snv_count[1] else None  # rename....
    a.append((gene_name, gene_snv_count[1], gene_intersection_count[1], ratio))
gene_ratios = list(filter(lambda x: x[1] != 0, a))
sorted(gene_ratios, key=lambda x: x[3], reverse=True)[:20]

[('ADNP_HUMAN', 1, 1, 1.0),
 ('ASH2L_HUMAN', 1, 1, 1.0),
 ('CBLL2_HUMAN', 1, 1, 1.0),
 ('DCP1A_HUMAN', 1, 1, 1.0),
 ('FOXJ2_HUMAN', 1, 1, 1.0),
 ('HBP1_HUMAN', 1, 1, 1.0),
 ('HNRL1_HUMAN', 1, 1, 1.0),
 ('HNRPL_HUMAN', 1, 1, 1.0),
 ('KDM3A_HUMAN', 1, 1, 1.0),
 ('KLF6_HUMAN', 1, 1, 1.0),
 ('MCRS1_HUMAN', 1, 1, 1.0),
 ('NR0B1_HUMAN', 1, 1, 1.0),
 ('PRKDC_HUMAN', 1, 1, 1.0),
 ('RUVB2_HUMAN', 1, 1, 1.0),
 ('SUMO1_HUMAN', 1, 1, 1.0),
 ('TBG1_HUMAN', 1, 1, 1.0),
 ('TET1_HUMAN', 1, 1, 1.0),
 ('TET2_HUMAN', 1, 1, 1.0),
 ('THOC4_HUMAN', 1, 1, 1.0),
 ('WDHD1_HUMAN', 1, 1, 1.0)]

In [11]:
a = np.array(gene_ratios)
pd.DataFrame(dict(gene=a[:, 0], ratio=a[:, 3], snv_count=a[:, 1], intersections_count=a[:, 2])).to_csv('intersection_ratios.csv')

In [12]:
list(filter(lambda x: "HNF" in x[0], a))

[array(['HNF1A_HUMAN', '19', '11', '0.5789473684210527'], dtype='<U32'),
 array(['HNF1B_HUMAN', '4', '0', '0.0'], dtype='<U32'),
 array(['HNF4A_HUMAN', '2447', '1008', '0.41193297915815286'], dtype='<U32'),
 array(['HNF4G_HUMAN', '59', '13', '0.22033898305084745'], dtype='<U32')]

In [13]:
list(filter(lambda x: "FOX" in x[0], a))

[array(['FOXA1_HUMAN', '11855', '609', '0.05137072964993673'], dtype='<U32'),
 array(['FOXA2_HUMAN', '3170', '506', '0.15962145110410095'], dtype='<U32'),
 array(['FOXA3_HUMAN', '300', '107', '0.3566666666666667'], dtype='<U32'),
 array(['FOXJ2_HUMAN', '1', '1', '1.0'], dtype='<U32'),
 array(['FOXK2_HUMAN', '9984', '1026', '0.10276442307692307'], dtype='<U32'),
 array(['FOXM1_HUMAN', '54', '7', '0.12962962962962962'], dtype='<U32'),
 array(['FOXO1_HUMAN', '65', '3', '0.046153846153846156'], dtype='<U32'),
 array(['FOXO3_HUMAN', '4', '3', '0.75'], dtype='<U32'),
 array(['FOXP1_HUMAN', '1357', '168', '0.12380250552689757'], dtype='<U32'),
 array(['RFOX2_HUMAN', '892', '171', '0.19170403587443946'], dtype='<U32')]

In [14]:
list(filter(lambda x: "HEP" in x[0], a))

[]

In [15]:
"""tf_names = [tf_name[0] for tf_name in intersection_counts]  ## TODO
#print(tf_names)
ratio = [(tf_name, 
          significant_snv_counts[tf_name],
          intersection_counts[tf_name] / significant_snv_counts[tf_name] 
              if significant_snv_counts[tf_name] 
              else None)
         for tf_name in tf_names]
#ratio = [snv_count / max(intersection_count, 1e-30) for snv_count, intersection_count in zip(significant_snv_counts, intersection_counts)]
ratio_nonempty = dict(filter(lambda x: x[1] is not None, ratio.items()))
print(*sorted(ratio_nonempty.items(), key=lambda x: x[1]), sep='\n')"""

"tf_names = [tf_name[0] for tf_name in intersection_counts]  ## TODO\n#print(tf_names)\nratio = [(tf_name, \n          significant_snv_counts[tf_name],\n          intersection_counts[tf_name] / significant_snv_counts[tf_name] \n              if significant_snv_counts[tf_name] \n              else None)\n         for tf_name in tf_names]\n#ratio = [snv_count / max(intersection_count, 1e-30) for snv_count, intersection_count in zip(significant_snv_counts, intersection_counts)]\nratio_nonempty = dict(filter(lambda x: x[1] is not None, ratio.items()))\nprint(*sorted(ratio_nonempty.items(), key=lambda x: x[1]), sep='\n')"

In [16]:
dic = { 0: [3,6,4], 1: [1,1,2], 2: [3,5,3], 3: [1,7,4], 4: [1,3,3], 5: [4,4,6] }
dict(filter(lambda e: sum(e[1]) > 10, dic.items()))

{0: [3, 6, 4], 2: [3, 5, 3], 3: [1, 7, 4], 5: [4, 4, 6]}

In [17]:
df = pd.read_csv("chr_hg38_struct.tsv", delimiter='\t')
chr_names = df["UCSC-style-name"]
chr_len = df["Sequence-Length"]

print(sum(chr_len))
#dict(zip(chr_names, chr_len))

3088269832


In [18]:
for qual in ('A', 'B', 'C', 'D'):
    total_cistr_length = 0
    with open(f"/home/resecmo/vigg/cistrome_hg38/hg38_cistrome/HNF4A_HUMAN.{qual}.bed") as cistr_file:
        for line in cistr_file.readlines():
            sp = line.strip().split('\t');
            #print(sp)
            total_cistr_length += int(sp[2]) - int(sp[1])

    print(total_cistr_length, total_cistr_length /  sum(chr_len))

14082642 0.004560042601873268
9936925 0.0032176349673320903
7494373 0.0024267222126592987
373857 0.00012105710327710768


In [19]:
'630091\n'

'630091\n'